In [109]:
# Avoids reloading kernel while developing
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [110]:
from WickContractions.wick.contract import *
from PyCorrelationMatrixManager.correlation_matrix import *

import os
import sys 

from src.utilities import *

sys.path.append(os.path.join(os.path.expanduser('~'), "Code", "LQCD", "contraction_optimizer_pymodule"))

#pretty latex printing
from IPython.display import display, Math 
pprint = lambda o : display(Math(str(o)))

import ContractionOptimizerCPP as CO

## Operators

First we have to define the operators

In [111]:
from src.op_elementals import *

aOps=[mesonBaryonSource(
    mesonData={'isospin': {'ud': 1}, 'x': 'p_0', 'g': '\\Gamma^M'},
    baryonData={'isospin': {'uuu': 1}, 'x': 'p_0', 'g': '\\Gamma^B'},
                                      )]

cOps=[mesonBaryonSink(
    mesonData={'isospin': {'du': 1}, 'x': 'p_0', 'g': '\\Gamma^M'},
    baryonData={'isospin': {'uuu': 1}, 'x': 'p_0', 'g': '\\Gamma^B'},
                                      )]

In [112]:
for o in aOps:
    pprint(o)

<IPython.core.display.Math object>

In [113]:
for o in cOps:
    pprint(o)

<IPython.core.display.Math object>

In [114]:
corrMatrix = CorrelationMatrix(
    cops=cOps, aops=aOps, 
    dts=[t for t in range(32)], t0s=[t for t in range(32)], 
    cfg=0)

In [115]:
corrMatrix.contract()

In [116]:
pprint(corrMatrix.correlators[0].diagrams[0])

<IPython.core.display.Math object>

In [117]:
from WickContractions.laph.diagram import LDiagram

tst=LDiagram(copy.deepcopy(corrMatrix.correlators[0].diagrams[0]))

pprint(tst)

<IPython.core.display.Math object>

In [118]:
tst.create_m_blocks()
tst.create_b_blocks()

In [119]:
pprint(tst)

<IPython.core.display.Math object>

In [120]:
tst.create_hadron_blocks()
pprint(tst)

<IPython.core.display.Math object>

In [121]:
corrMatrix.laphify()

In [122]:
len(corrMatrix.get_all_diagrams())

24

In [123]:
for d in corrMatrix.get_all_diagrams():
    pprint(d)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [126]:
tensors,contractions=corrMatrix.correlators[0].diagrams[0].as_graph()
print(tensors)
print(contractions)

{0: 'M(p_0,t_i,t_f,\\Gamma^M)', 1: 'M(p_0,t_i,\\Gamma^M)', 2: 'B^*(p_0,t_i,t_f,\\Gamma^B)', 3: 'B(p_0,t_i,\\Gamma^B)'}
{'3|2': [[2, 0], [0, 2]], '3|0': [[1, 1]], '2|1': [[1, 0]], '1|0': [[1, 0]]}


In [127]:
def contraction_as_COgraph(contractions):
    data=[]
    for cname, cindices in contractions.items():
        [l,r] = cname.split('|')
        data.append(( (int(l),int(r)), set(tuple(t) for t in cindices) ))

    return CO.Graph(data)

In [128]:
global_tensors={}
global_tidx=0   
all_diagrams=[]
duplicate_tensors_fix=[]

# for the temporary fix, we have to through all the diagrams twice
# first we have to get the global tensors list
for i,d in enumerate(corrMatrix.get_all_diagrams()[0:1000]):
    tensors, contractions = d.as_graph()
    
    for t in tensors.values():
        if t not in global_tensors:
            global_tensors[t]=global_tidx
            global_tidx+=1

for tensor in global_tensors.copy().keys():
    global_tensors[tensor+"-B"]=global_tidx
    global_tidx+=1

for i,d in enumerate(corrMatrix.get_all_diagrams()[0:1000]):
    tensors, contractions = d.as_graph()

    cograph = contraction_as_COgraph(contractions)

    gtensors=[]
    duplicate_tensors_fix.append({})
    for t in tensors.values():
        if global_tensors[t] in gtensors:
            gtensors.append(global_tensors[t+"-B"])
        else:
            gtensors.append(global_tensors[t])

    #print(gtensors)
    all_diagrams.append(CO.Diagram(cograph,gtensors))

In [129]:
CO.ContractionCost.setDilutionRange(16)
optimizer = CO.ContractionOptimizer(all_diagrams)
optimizer.tune()

Diagram  1 / 24
Diagram  2 / 24
Diagram  3 / 24
Diagram  4 / 24
Diagram  5 / 24
Diagram  6 / 24
Diagram  7 / 24
Diagram  8 / 24
Diagram  9 / 24
Diagram  10 / 24
Diagram  11 / 24
Diagram  12 / 24
Diagram  13 / 24
Diagram  14 / 24
Diagram  15 / 24
Diagram  16 / 24
Diagram  17 / 24
Diagram  18 / 24
Diagram  19 / 24
Diagram  20 / 24
Diagram  21 / 24
Diagram  22 / 24
Diagram  23 / 24
Diagram  24 / 24


In [130]:
print("Contractions | GlobalTensor | ResultTensorIdx")
for c in optimizer.getCompStepList():
    print("{:<30} | {:^8} | {:^5}".format(str(CO.Graph.decodeElement(c[0])), str(c[1]), str(c[2])))

Contractions | GlobalTensor | ResultTensorIdx
{(0, 2), (2, 0)}               |  (2, 3)  |   4  
{(0, 1)}                       |  (0, 1)  |   5  
{(0, 1), (1, 0)}               |  (4, 5)  |   6  
{(0, 2), (2, 1)}               |  (2, 3)  |   7  
{(0, 1), (1, 0)}               |  (5, 7)  |   8  
{(2, 1), (0, 0)}               |  (2, 3)  |   9  
{(0, 1), (1, 0)}               |  (5, 9)  |  10  
{(0, 1), (2, 0)}               |  (2, 3)  |  11  
{(0, 1), (1, 0)}               | (5, 11)  |  12  
{(0, 1), (2, 2)}               |  (2, 3)  |  13  
{(0, 1), (1, 0)}               | (5, 13)  |  14  
{(2, 2), (0, 0)}               |  (2, 3)  |  15  
{(0, 1), (1, 0)}               | (5, 15)  |  16  
{(1, 1), (0, 2)}               |  (2, 3)  |  17  
{(0, 1), (1, 0)}               | (5, 17)  |  18  
{(1, 0), (0, 2)}               |  (2, 3)  |  19  
{(0, 1), (1, 0)}               | (5, 19)  |  20  
{(0, 1), (1, 0)}               |  (2, 3)  |  21  
{(0, 1), (1, 0)}               | (5, 21)  |  22  
{(1,

In [131]:
for d in optimizer.getDiagramList():
    print(d.getResultIdList())

[6]
[8]
[10]
[12]
[14]
[16]
[18]
[20]
[22]
[24]
[26]
[28]
[29, 30]
[30, 31]
[30, 32]
[30, 33]
[30, 34]
[30, 35]
[37]
[39]
[41]
[43]
[45]
[47]
